 # INM430 - Tiny DS Project Progress Report


***

** Student Name: ** Elisa Pessoni de Padua Chimenton

** Project Title:** How positive reviews and neighbourhood have influence in Airbnb's price 

***


# Importing libraries and loading data

In [503]:
#libraries for text processing
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from scipy.stats import norm

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, classification_report, make_scorer,r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score

#usual imports
%matplotlib inline
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import dask.dataframe as dd
from collections import defaultdict
# Variables for colours
BNB_BLUE = '#007A87'
BNB_RED = '#FF5A5F'
BNB_DARK_GRAY = '#565A5C'
BNB_LIGHT_GRAY = '#CED1CC'

#load data
reviews = pd.read_csv(r'C:\Users\44746\Documents\Data Science\reviews.csv.gz', compression='gzip', header=0, sep=',', quotechar='"',low_memory=False)
calendar = pd.read_csv(r'C:\Users\44746\Downloads\calendar.csv.gz', compression='gzip', header=0, sep=',', quotechar='"',low_memory=False)
listing = pd.read_csv(r'C:\Users\44746\Documents\Data Science\listings.csv.gz', compression='gzip', header=0, sep=',', quotechar='"',low_memory=False)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\44746\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\44746\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\44746\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [504]:
# verify dataframe dimensions features=columns entries=data
print("Features:\t{}\nEntries: \t{}".format(listing.shape[1], listing.shape[0]))
pd.set_option('display.max_columns', len(listing.columns)) # To view all columns
pd.set_option('display.max_rows', 100)
listing.head()

Features:	106
Entries: 	85273


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,11551,https://www.airbnb.com/rooms/11551,20190914144451,2019-09-15,Arty and Bright London Apartment in Zone 2,Unlike most rental apartments out there my fla...,"Amenities Bedding: 1 Double bed, 1 living room...",Unlike most rental apartments out there my fla...,family,Not even 10 minutes by metro from Victoria Sta...,No Smoking (very strict) Check-in time is afte...,Tons of buses (24hrs) go into central London f...,Guest will have access to the entire apartment,No interaction with guests as you book the ent...,No Smoking (very strict) No pets are allowed i...,NaN,NaN,https://a0.muscache.com/im/pictures/1d720898-c...,NaN,43039,https://www.airbnb.com/users/show/43039,Adriano,2009-10-03,"London, England, United Kingdom","Hello, I'm a friendly Italian man with a very ...",within an hour,100%,NaN,t,https://a0.muscache.com/im/pictures/user/2beb9...,https://a0.muscache.com/im/pictures/user/2beb9...,Brixton,3.0,3.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"London, United Kingdom",Brixton,Lambeth,NaN,London,NaN,SW9 8DG,London,"London, United Kingdom",GB,United Kingdom,51.46225,-0.11732,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Paid par...",NaN,$88.00,$645.00,"$2,350.00",$400.00,$30.00,2,$20.00,2,180,2,2,180,180,2.0,180.0,4 days ago,t,19,42,72,344,2019-09-15,184,1,2010-03-21,2019-07-10,92.0,9.0,9.0,10.0,10.0,9.0,9.0,f,NaN,NaN,t,f,strict_14_with_grace_period,f,t,2,2,0,0,1.59
1,13913,https://www.airbnb.com/rooms/13913,20190914144451,2019-09-15,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,For art lovers I can give guest my Tate Member...,The flat only a 10 minute walk to Finsbury Par...,Guest will have access to the self catering ki...,I like to have little chats with my guest over...,I'm an artist and have my artwork up on the wa...,NaN,NaN,https://a0.muscache.com/im/pictures/ffb507b7-9...,NaN,54730,https://www.airbnb.com/users/show/54730,Alina,2009-11-16,"London, England, United Kingdom",I am a Multi-Media Visual Artist and Creative ...,within an hour,100%,NaN,f,https:/

In [505]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,11551,30672,2010-03-21,93896,Shar-Lyn,"The flat was bright, comfortable and clean and..."
1,11551,32236,2010-03-29,97890,Zane,We stayed with Adriano and Valerio for a week ...
2,11551,41044,2010-05-09,104133,Chase,Adriano was a fantastic host. We felt very at ...
3,11551,48926,2010-06-01,122714,John & Sylvia,We had a most wonderful stay with Adriano and ...
4,11551,58352,2010-06-28,111543,Monique,I'm not sure which of us misunderstood the s...


**Comments:** This work will mainly have a overview about listings dataset in first part of analysis, next step will also look at reviews dataset comparing these reviews with location, then look at description title of listings and last evaluate how amenities influencing in price.

# Data cleaning

Dropping all columns that seems not relevant to this analysis as listing_url,street, city, state because will be analysed London only. Street don't need to be included it has been analysed by neighbourhood.

In [506]:
#drop some columns that seems unnecessary
cols_to_drop = ['listing_url','host_since','scrape_id','host_has_profile_pic',
                 'street', 'city','state', 'zipcode','scrape_id', 'last_scraped', 'space',  'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 
                'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url',
                'host_neighbourhood', 'host_verifications', 'calendar_last_scraped', 'host_acceptance_rate']
listing = listing.drop(cols_to_drop, axis=1)

In [507]:
#check data types after drop some columns
listing.dtypes

id                                                int64
name                                             object
summary                                          object
description                                      object
experiences_offered                              object
host_response_time                               object
host_response_rate                               object
host_is_superhost                                object
host_listings_count                             float64
host_total_listings_count                       float64
host_identity_verified                           object
neighbourhood                                    object
neighbourhood_cleansed                           object
neighbourhood_group_cleansed                    float64
market                                           object
smart_location                                   object
country_code                                     object
country                                         

In [508]:
#check missing values
listing.isnull().sum().sort_values(ascending=False)

neighbourhood_group_cleansed                    85273
jurisdiction_names                              85271
license                                         85100
square_feet                                     84859
monthly_price                                   80144
weekly_price                                    78082
security_deposit                                29652
host_response_time                              24822
host_response_rate                              24822
review_scores_checkin                           22781
review_scores_value                             22781
review_scores_location                          22780
review_scores_accuracy                          22733
review_scores_communication                     22726
review_scores_cleanliness                       22720
review_scores_rating                            22682
cleaning_fee                                    21529
last_review                                     21003
first_review                

In [509]:
# Remove more columns
listing = listing.drop(['market','neighbourhood_group_cleansed','square_feet', 'country_code','availability_365','license','jurisdiction_names','instant_bookable', 
                        'calculated_host_listings_count', 'require_guest_profile_picture', 'require_guest_phone_verification', 'is_business_travel_ready', 'has_availability', 'medium_url', 'xl_picture_url'], 1)


KeyError: "['medium_url' 'xl_picture_url'] not found in axis"

In [ ]:
# Replacing columns with f/t with 0/1
listing.replace({'f': 0, 't': 1}, inplace=True)

# Plotting the distribution of numerical and boolean categories
listing.hist(figsize=(20,20));

**Comments:** The cells above shows types of variables, quantity of missing number to be handle and distribution of categorical, boolean values in order understand better data and presence of outliers.

In [ ]:
#Distribution of listings type
listing.groupby('room_type').count()['id']

**Comment:** Number of entire home/apt is the highest, followed by private room, however hotel room and shared room is much lower. Seems that people use to rent their house when they are out as way of increase their earnings.

In [ ]:

listing.groupby(['neighbourhood_cleansed', ]).count()['id']

In [ ]:
corr = listing.corr()
print(corr.head())

plt.figure(figsize = (10,10))

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True)


In [ ]:
listing[['accommodates', 'bathrooms', 'bedrooms', 'beds']].hist(figsize=(8,6));

**Comment:** Check quantity of listing per boroughs of London, Westminster and Tower Hamlets have the majority of listings, despite the listings are distributed around whole London.

In [ ]:
#using commas to separate amenities strings
listing.amenities = listing.amenities.str.replace("[{}]", "").str.replace('"', "")
listing['amenities'].head()

**Comment:** It's known that some amenities is more important and common than other is the last part of this analysis selection of more important amenities will be made and also they can contribute in higher or lower prices.

# Convert price to numerical variable

In [ ]:
#Convert the prices to numerical variable
price_clean = listing['price'].str.replace("$","").str.replace(',','').astype(float)
listing['price'] = price_clean

**Comment:** Price will be converted to an interger because currently they are as string including currency sign.

In [ ]:
plt.figure(figsize=(10,5))
listing.groupby('accommodates').price.median().plot(kind='bar')
plt.title('Median price of Airbnbs accommodating different number of guests', fontsize=14)
plt.xlabel('Number of guests accommodated', fontsize=13)
plt.ylabel('Median price (£)', fontsize=13)
plt.xticks(rotation=0)
plt.xlim(left=0.5)
plt.show()

# Price distribution in listings

**Comment:** Price is the target variable in this analysis for this reason is fundamental to understand well how it is distributed before start data preparation. Below 3 different histograms was created, one using all the data and the last 2 setting smaller number of bins.  

In [ ]:
#Price distribution in listings
#Function to plot histogramas
# Input: n(interger for subplots)
#        titles (list of titles)
#        ranges (list of values)
# Output: None

def plot_hist(n, titles, ranges):
    fig, ax = plt.subplots(n, figsize = (8, 7.5))
    for i in range(n):
        d, bins, patches = ax[i].hist(ranges[i], 50, density = 1,color= BNB_RED,  alpha = 0.85)
        ax[i].set_title(titles[i])
        ax[i].set_xlabel("Daily Listing Price in Pounds")
        ax[i].set_ylabel("Frequency")
    plt.tight_layout()
    plt.show()
prices = listing['price']

# Plot histograms of price distribution
plot_hist(3, ['Distribution of Listing Prices: All Data', 'Distribution of Listing Prices: £0 -  £1000', 
               'Distribution of Listing Prices: £0 - £350'], [prices, prices[prices <= 1000], prices[prices < 350]])

# Distribution of listing by geographical location

In [ ]:
# Distribution of listing by geographical location
# Outline price buckets
intervals = [0,100,300, 1000]
leg_labels = []

# Get Labels for legend
for i in range(0,len(intervals) - 1):
    if i == len(intervals) - 2:
        leg_labels.append('£{}+'.format(intervals[i]))
    else:
        leg_labels.append("£{}- £{}".format(intervals[i], intervals[i+1]))    

buckets = []

# Divide up into price buckets
for i in range(0, len(intervals) - 1):
    buckets.append(listing[(prices > intervals[i]) & (prices < intervals[i+1])])

colors = [BNB_LIGHT_GRAY, BNB_BLUE, BNB_RED]
alphas = [0.85, 0.85, 0.85]  



# Plot listings on scatterplot
plt.figure(figsize=(15, 15))
for i in range(0, len(buckets)):
    plt.scatter(buckets[i]['longitude'], buckets[i]['latitude'], alpha = alphas[i], c=colors[i], s=25)
    
plt.title('London Airbnb Listings')
plt.xlabel('Longitute')
plt.ylabel('Latitude')
plt.legend(labels=leg_labels, loc = 'best')
plt.xlim(-0.5,0.3)
plt.ylim(51.25,51.70)

plt.show()

In [ ]:
# Plot graphs neighbourhood vs price
fig, (axis1) = plt.subplots(1, figsize=(20,20))

sns.barplot(x='price', y='neighbourhood_cleansed', data=listing,orient= 'index', ax=axis1)

# Data preparation

**Create new column price per person**
Airbnb price are listed per night and when allowed to include some guest there is a charge fee per person.This means there are large differences in prices due to number of guests allowed per acommodation, what makes it difficult to compare. To facilitate this comparation and focusing on how tourists normally search for accomodation, a new column price per person (per night) will be created. 
To calculate price per person (per night) column, first is calculated the normalized total price per night costs corresponding to house's capacity and then divide by house's capacity as shown below.

In [ ]:
#The columns needed for obtaining prices per person per night
listing[['price', 'accommodates', 'guests_included', 'extra_people']].head()

In [ ]:
#listing price including outliers
listing['price'].describe()

In [ ]:
#The columns needed for calculation do not have any missing values
listing[['price', 'accommodates', 'guests_included', 'extra_people']].isna().sum()

In [ ]:
#Obtain the number of people that can be accommodated but are not included in the price
extra_space = (listing['accommodates'] - listing['guests_included'])
extra_space.head(5)

In [ ]:
#Due to inconsistencies in the imported data this number is negative in some rare cases, in this cases will be add 0.
extra_space = extra_space.apply(lambda x: 0 if x<0 else x)
#Calculate the extra costs
extra_price = extra_space*listing['extra_people'].str.replace("$","").str.replace(',','').astype(float)

In [ ]:
#Obtain the price at full capacity and calculate price per person per night
listing['price_pp'] = (price_clean + extra_price) / listing['accommodates']

In [ ]:
listing['price_pp'].describe()

In [ ]:
#Distribution of prices with all outliers
plt.rcParams['figure.figsize'] = [12, 5]
sns.distplot(listing['price']);
plt.xlabel('Price per Night');
plt.ylabel('Frequency');
plt.title('Prices of Airbnb listings in London');

In [ ]:
#Distribution of prices from column price_pp
plt.rcParams['figure.figsize'] = [12, 5]
sns.distplot(listing['price_pp']);
plt.xlabel('Price per person per Night');
plt.ylabel('Frequency');
plt.title('Prices of Airbnb listings per person per night in London');

In [ ]:
# price
listing.price.plot(kind='box', vert=False, figsize=(15, 3),)
plt.show()

# check price over £1000
print("\nPrice over £1000:")
print("{} entries".format(len(listing[listing.price > 1000])))
print("{:.4f}%".format((len(listing[listing.price > 1000]) / listing.shape[0])*100))

# listing.price.plot(kind='box', vert=False, xlim=(0,1300), figsize=(15,3));

# Remove outliers

In [ ]:
#Remove outliers using Tukey rule
IQR = listing['price_pp'].quantile(0.75) - listing['price_pp'].quantile(0.25)
lower_bound = listing['price_pp'].quantile(0.25) - 3*IQR
upper_bound = listing['price_pp'].quantile(0.75) + 3*IQR
lower_bound, upper_bound

In [ ]:
#Override lower bound with a positive number 
lower_bound = 5
listing = listing[(listing['price_pp']>lower_bound) & (listing['price_pp']<upper_bound)]

In [ ]:
# verify dataframe dimensions features=columns entries=data
print("Features:\t{}\nEntries: \t{}".format(listing.shape[1], listing.shape[0]))

# Distribution of price per room type.

In [ ]:
#Distribution of prices after removing outliers
plt.rcParams['figure.figsize'] = [12, 6]
#plt.rc('axes', labelsize = 14) 
#plt.rc('font', size=14)   
sns.distplot(listing['price_pp'][listing["room_type"]=='Entire home/apt'],bins = np.arange(0,108,5));
plt.xlabel('Price per Person per Night (£)');
plt.ylabel('Frequency');
plt.title('Distribution of prices of Airbnb apartments in London (without outliers)');
#plt.savefig('Price Distribution')

In [ ]:
# We'll make use of the qqplot function from the stasmodel package
import statsmodels.api as sm
#This contains theoretical distributions
from scipy import stats

sm.qqplot(listing['price_pp'], dist=stats.norm, line = 'r')
plt.show()



In [ ]:
# We'll use the np.log() function and use it as a parameter to the pandas .apply() function
loggedprice_pp=listing['price_pp'].apply(np.log)
fig = sm.qqplot(loggedprice_pp, stats.norm, line = 'r')
plt.show()

plt.suptitle("log(" + 'price_pp' + ")")
plt.hist(loggedprice_pp, 20)

In [ ]:
listing['price_pp'].describe()


In [ ]:
#Fill the missing names (Airbnb title) in the listings dataset
listing['name'].fillna('No Name', inplace = True)

In [ ]:
#Fill in the missing comments in the reviews dataset
reviews['comments'].fillna('No Comment', inplace = True)

In [ ]:
#Check reviews dataset that will be used to answer questions 3 and 4
reviews.head()

# Data Modeling

In the following analysis, listings in different London boroughs are calculated. The average price in each of them are compared. This simple analysis is repeated on different subsets of the original dataset, for this reason will be implemented a function to facilitate to repeat process.

In [ ]:
def price_neighborhood_cleansed(df, filters = None):
    """" Calculates and visualizes the average price per person per night in each neighborhood. 
    Filters can be applied so the analysis can be repeated on different subsets of the original dataframe.
    
    Parameters
    ----------
    df : DataFrame, detailed data of Airbnb listings including their price per person per night
    filters: dict, column (str) and values (list) pairs used to filter the dataset 
    
    Returns
    -------
    avg_prices_df : DataFrame, average price per person per night in each neighborhood
    """
    
    #Empty Filter
    bools = [True]*df.shape[0]
    
    #Combine all the filters
    if filters:
        for column, value in filters.items():
            additional_filter = (df[column].isin(value))
            bools = bools & additional_filter

    #Apply the filters
    df_filtered = df[bools]
        
    #Calculate average prices in different neighborhoods
    avg_prices = df_filtered.groupby(['neighbourhood_cleansed'])['price_pp'].mean().sort_values(ascending = False)
    
    #Store them in a dataframe
    avg_prices_df = pd.DataFrame(avg_prices).reset_index()

    #Visualize the results
    colors = sns.color_palette('muted')
    sns.barplot(y = 'neighbourhood_cleansed', x = 'price_pp', data = df_filtered, 
                order = avg_prices.index, orient='h', color= colors[3], zorder= 2)
    plt.xlabel('Average Price per Night (£)')
    plt.ylabel('')
    plt.title('Average Prices in London Neighborhoods')
    plt.subplots_adjust(left=0.35, right=0.9, top=0.95, bottom=0.07)
    plt.grid(axis = 'x', linestyle = '--', zorder=0)

    return avg_prices_df

In [ ]:
#Average prices for different neighborhoods, all listings
plt.rcParams['figure.figsize'] = [12, 8]
all_avg = price_neighborhood_cleansed(listing);
#plt.savefig('neighbourhood')

In [ ]:
#Average prices for different neighborhoods, entire home/apartment listings only
#plt.rc('axes', labelsize = 13) 
#plt.rc('font', size = 13)   
entirehouse_avg = price_neighborhood(listing, {'room_type': ['Entire home/apt']});
#plt.savefig('Neighbourhoods')

In [ ]:
#Average prices for different neighborhoods, entire 2-person home/apartment listings only
entirehouse2_avg = price_neighborhood(listing, {'room_type': ['Entire home/apt'], 'accommodates':[2]});

In [ ]:
#Average prices for different neighborhoods, private rooms only
priv_room_avg = price_neighborhood(listing, {'room_type': ['Private room']});

In [ ]:
#Compare entire home prices and private room prices
compare = private_avg.merge(entirehouse_avg, on = 'neighbourhood_cleansed')
compare['price_pp_x'] - compare['price_pp_y']

In [ ]:
#Correlation between prices of entire homes and private room prices
compare.corr()

# Compare the percentage of positve guests reviews about location with price

**Text wrangling and pre-processing - comments:** One of the most commons text wrangling and pre-processing for textual data has been used to analyse guests reviews: remove special characters and symbols that are usually non-alphanumeric and can add extra noise in unstructured texts. Regular expressions(regexes) was used to remove them. Another one is remove stopwords (articles, conjunctions) as a, an, the and transform sentences in words.There is no universal stopword list but here it is been used nltk-standard English language stopwords list.For all these steps was created a function that d the cleaning first and then define the positive words and flag it as false when it doesn't have positive words or the second sentence exclude the first one using not.

In [ ]:
def positive_sentiment(text):
    """" Splits the input text into sentences, and determines whether there exists
    a sentence that includes a positive comment about the location rented.
    
    Parameters
    ----------
    text : str, analyzed user review
    
    Returns
    -------
    positive_flag: boolean, indicates whether the review includes a positive comment about the location
    """
    
    #Split the input text into sentences
    sentences = sent_tokenize(text)
    
    #Remove special characters, clean sentences and keep the ones that include the word 'location'
    sentences_without_carac = [re.sub(r'[^a-zA-Z0-9]', ' ', sent.lower()).strip() for sent in sentences 
                       if 'location' in sent.lower()]
    
    #In case there are more sentences describing location, merge them together as one long sentence
    one_string = " ".join(sentences_without_carac)
    
    #Tokenize the sentence into words
    location_words = set(word_tokenize(one_string))
    
    #Define words that represent positive sentiment
    positive_words = ['central', 'good', 'nice', 'best', 'great', 'perfect', 'excellent', 
            'superb', 'amazing', 'awesome', 'incredible', 'wonderful']
    
    #Determine whether the (merged) sentence about the location includes any of the positive words and 
    #excludes the first sentence using word 'not' 
    if ((len(location_words.intersection(positive_words))  >=1) & ('not' not in location_words)):
        positive_flag = True
    else: 
        positive_flag = False
    
    return positive_flag

In [ ]:
positive_sentiment('The location was just excellent.')

In [ ]:
positive_sentiment('The apartments is nice. But the location is not so great')

In [ ]:
#Create a new column that identifies positive location reviews
reviews['positive_location_review'] = reviews['comments'].apply(positive_sentiment)

In [ ]:
#Total umber of positive location reviews
reviews['positive_location_review'].sum()

In [ ]:
#Total relative number of positive location reviews
reviews['positive_location_review'].sum()/reviews.shape[0]

In [ ]:
#Count the number of reviews per listing and number of positive location reviews per listing
review_counts = reviews.groupby('listing_id')['positive_location_review'].count()
positive_review_counts = reviews.groupby('listing_id')['positive_location_review'].sum()

In [ ]:
#Store the information into a dataframe
review_counts_df = pd.concat([review_counts, positive_review_counts],axis=1)
review_counts_df.columns = ['total_count','positive_count']
review_counts_df.index.name = 'id'

In [ ]:
listing_data = listing[['id','neighbourhood_cleansed']]
additional_filter = (listing['room_type'] == 'Entire home/apt')
listing_data = listing_data[additional_filter]
merged_df = listing_data.merge(review_counts_df, on ='id')

**Comparing guests reviews with location - comments:** It was used text processing techniques above to identify reviews with such positive comments, and then obtain, through mapping (guest reviews from the reviews dataset are linked to the corresponding listings in the listings dataset) and aggregation (from individual listings to neighborhoods_cleansed), compared with the percentage of positive location reviews in each borough.
This analysis observe if the users' opinions about the location are correlated with the location-driven price differences, and potentially identify London boroughs that seems under/overpriced given their location.

In [ ]:
merged_df.head(10)

In [ ]:
merged_df.shape

In [ ]:
#Calculate the total number of reviews per neighborhood and total number of positive location reviews per neighborhood
review_counts_nh = merged_df.iloc[:,1:].groupby('neighbourhood_cleansed').sum()
review_counts_nh['positive_location_ratio'] = 100*review_counts_nh['positive_count'] / review_counts_nh['total_count']

In [ ]:
#Order the neighborhoods with respect to the ratio of reviews that positively comment on the location
review_counts_nh.sort_values('positive_location_ratio', ascending = False)

In [ ]:
final_results = review_counts_nh.merge(homeapt_avg, on ='neighbourhood_cleansed').sort_values('positive_location_ratio', 
                                                                                          ascending = False)
final_results

In [ ]:
#Calculate the correlation coefficient between the average prices and the relative number of positive location reviews
print(final_results[['positive_location_ratio', 'price_pp']].corr())
print(final_results[['positive_location_ratio', 'price_pp']].corr('kendall'))

In [ ]:
#Show linear relationship with a scatter plot using seaborn
plt.rcParams['figure.figsize'] = [16, 12]
plt.rcParams['lines.color'] = 'b'
#plt.rc('axes', labelsize = 12) 
#plt.rc('font', size=14) 
sns.regplot('positive_location_ratio', 'price_pp', data = final_results, color="g");
plt.xlabel('Positive Location Reviews (%)');
plt.ylabel('Average Price per Person per Night (£)');
plt.grid( linestyle = '-')

add_text = [1, 2, 3, 4, 5, 6, 8, 9,10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 28, 31, 32]
horizontal = ['left','left','right','left','left','left','left','left','left','left','left','right','left','right','left','left','left','left','left','left','left','right','left','right','left','left','left','left']
vertical = ['bottom','bottom','bottom','bottom','top','top','top','top','top','bottom','top','bottom','bottom','bottom','bottom','top','top','top','top','bottom','bottom','bottom','bottom','bottom','top','top','top']
for i in range(len(add_text)):
    plt.text(final_results['positive_location_ratio'][add_text[i]],
             final_results['price_pp'][add_text[i]], 
             final_results['neighbourhood_cleansed'][add_text[i]],
             horizontalalignment = horizontal[i],
             verticalalignment = vertical[i])
#plt.savefig('Location Reviews')

**Correlation -Comments:** The function regplot in seaborn was used to visualise linear relationship.
The regression plot intend to add a visual guide to highlight patterns in average price per night and positive locations from guests reviews, showing what was expected that the price is strongly correlated to the ratio of positive location reviews and most of cases price would be right not under/overpriced, in few cases as Greenwich and Kigston uopn Thames seems under priced and Kingston and Chelsea and City of London a bit overpriced hwever they also presented the best ration of positive location.
In order to not clutter the image some data points was left without borough's names but the relation of all boroughs can be seen in final_results above.

# Identifying adding-value features from listings description titles

**Adding value features - comments:** Location was proved above to be a factor impacting Airbnb prices, but should have more covered aspects.This part seeks to identify features that add value to Airbnb entire house or apartments, by analyzing the information contained in listings’ textual descriptions.For each specific feature of interest, it will be performed a word search through the Airbnb listings’ titles and split the input data-set into two subsets, separating listings with and without specific feature advertised, allowing compare the distribution of prices between these two datasets and construct a confidence interval for the difference of means. In this way would identify features that may have a significant impact on the prices.


In [ ]:
#Checking listings description titles
display(listing['name'])
summary = listing.dropna(subset=['name'], axis=0)['name']
all_summary = " ".join(s for s in summary)
print("Total of words: {}".format(len(all_summary)))


In [ ]:
#Produce wordCloud from listings description titles
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords,
                      background_color="black",
                      width=1600, height=800).generate(all_summary)
# plot image
fig, ax = plt.subplots(figsize=(10,6))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()

plt.imshow(wordcloud);



**Comments:** Interesting! This wordcloud shows that the high demand or may search for accomodation are for double bedroom, doble room, central London as most of hosts include it in their descriptions title as showed in wordcould above but this is not interesting for this analysis so next step will be stop this common words that seems do not add any value to houses, to try to find the most interesting ones.It also prove that this analysis will have more value for entire house or apartments because normally shared rooms don't offer more features than bedroom itself.


In [ ]:
#Including commons words from analysis above
stopwords.update(["double", "room", "Central", "London", "double", "bedroom", "bed", "flat", "bedroom", "flat", "house", "Apartment"])
# gerar uma wordcloud
wordcloud = WordCloud(stopwords=stopwords,
                      background_color="black",
                      width=1600, height=800).generate(all_summary)
# mostrar a imagem final
fig, ax = plt.subplots(figsize=(10,6))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()

plt.imshow(wordcloud);


**Comments:** Now without the commom words from the wordCloud above seems more with what was expected for this analysis, the keywords Cosy, Lovely, Large, Spacious, Modern, Garden, Balcony, Penthouse, Stylish will be examined to check if they add any value to houses and apartments.

In [ ]:
def mean_difference(df, keyword, text_column = 'name', filters = None, plot_text = "", confidence_level = 90):
    """ Split the input dataset into two subsets, separating listings that have a specific keyword
    in the name or description and the ones that do not. Compare the distribution of prices between the
    two datasets and construct a confidence interval for the difference in means.
    
    Parameters
    ----------
    df : DataFrame, detailed data of Airbnb listings including their price per person per night
    keyword : str, specific keyword used to separate the dataset
    text_column : str, analyzed column with textual descriptions of properties, 'name' or 'description'
    filters : dict, column(str) - value(list) pairs used to filter the original dataset
    plot_text : str, text used in the visualization
    confidence_level : int, confidence level used to construct confidence intervals
    
    Returns
    -------
    None
    
    """
    #Empty filter
    bools = [True]*df.shape[0]
    
    #Apply filters to obtain the desired subset
    if filters:
        for column, value in filters.items():
            additional_filter = (df[column].isin(value))
            bools = bools & additional_filter
    df = df[bools]
    
    # Check each name if it includes the input keyword 
    keyword_flag = df[text_column].apply(lambda x: 1 if keyword in x.lower() else 0)
    
    # Split the input dataframe into two sets - one with the keyword in the name and one without
    prices_keyword = df['price_pp'][keyword_flag == 1]
    prices_other = df['price_pp'][keyword_flag == 0]
    
    #Plot the distribution
    bins = np.arange(0,200,5)
    sns.distplot(prices_keyword, bins, kde = True, norm_hist = True)
    sns.distplot(prices_other, bins, kde = True, norm_hist = True, color = 'green')
    plt.title(f'Keyword: "{keyword}"')
    plt.xlabel('Price per Person per Night (£)')
    
    #Text in the legend
    if plot_text == "":
        plot_text = keyword
    
    plt.legend([f'Listings advertised as {plot_text}','Other'])
    
    #Calculate the confidence interval for the mean difference
    m1 = prices_keyword.mean()
    m2 = prices_other.mean()
    
    s1 = prices_keyword.std()
    s2 = prices_other.std()
    
    n1 = len(prices_keyword)
    n2 = len(prices_other)
    
    se = np.sqrt((s1 ** 2)/n1 + (s2 ** 2)/n2)
    
    #Sample mean difference
    stat = m1-m2
    
    #Inverse CDF
    p = 1 - (1 - confidence_level/100)/2
    critical = norm.ppf(p)
    
    #Confidence interval
    #lower_bound = stat - se * critical
    #upper_bound = stat + se * critical
    
    output_str = f'Difference in means: {stat:.2f} (+/- {se * critical:.2f})'
    
    props = dict(boxstyle='round', facecolor='white', alpha=1)
    plt.text(-5, 0.001, output_str , fontsize=18,
        verticalalignment='bottom', bbox = props)

    
    #print(f'Average price difference : {stat:.2f} +/- {se * critical:.2f}')

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]
mean_difference(listing, 'luxury', filters = {'room_type':['Entire home/apt']})
#plt.savefig('luxury')

In [ ]:
mean_difference(listing, 'stylish',  filters = {'room_type':['Entire home/apt']})
#plt.savefig('stylish')

In [ ]:
mean_difference(listing, 'cosy',  filters = {'room_type':['Entire home/apt']})
#plt.savefig('stylish')

In [ ]:
mean_difference(listing, 'garden', plot_text = 'having a garden', filters = {'room_type':['Entire home/apt']})
#plt.savefig('garden')

In [ ]:
mean_difference(listing, 'modern', filters = {'room_type':['Entire home/apt']})
#plt.savefig('modern')

In [ ]:
mean_difference(listing, 'bright', filters = {'room_type':['Entire home/apt']})
#plt.savefig('modern')

In [ ]:
mean_difference(listing, 'balcony', filters = {'room_type':['Entire home/apt']})
#plt.savefig('modern')

In [ ]:
mean_difference(listing, 'rooftop', filters = {'room_type':['Entire home/apt'] })
#plt.savefig('rooftop')


In [ ]:
mean_difference(listing, 'spacious', filters = {'room_type':['Entire home/apt']})
#plt.savefig('modern')

In [ ]:
mean_difference(listing, 'river', filters = {'room_type':['Entire home/apt']}, plot_text = 'view to river Thames')
#plt.savefig('river')

Entire houses or apartments described as luxury are on average £8 more expensive than other Airbnb accommodation without this description.However, properties advertised with keywords as Cosy, Large, Lovely, Spacious, Modern seems do not increase any value sometimes they are below the average, they seems to used as a way to attract people to look at their house when they do not have specials features to describe.
Another keyword that added value to title was Rooftop increasing on average £5.


# Amenities

In [ ]:
count_amenities = listing['amenities'].value_counts()
count_amenities

In [ ]:
listing.amenities = listing.amenities.str.replace("[{}]", "").str.replace('"', "")
listing['amenities'].head()

In [ ]:
#Essentials amenities by listings
pd.Series(np.concatenate(listing['amenities'].map(lambda amns: amns.split(","))))\
    .value_counts().head(20)\
    .plot(kind='bar')
ax = plt.gca()
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=12)
plt.show();

**Comment:** In the list of all amenities showed above, can be deducted that some amenities can be more important than others (e.g. a balcony is more likely to increase price than a fax machine), and some are likely to be fairly uncommon (e.g. 'Electric profiling bed') but also is important hosts know wich is the most commom amenities because they can miss the opportunity to rent their homes without it (e.g. Wifi is expected for guests and seems be present at most of listings what shows probably this don't increase price but hosts can lose without it.)But in cases as iron or hair dryer , shampoo that is not expesive items if hosts do not have it can think about to buy it and increase their rent opportunities.

In [ ]:
corr = listing.corr()
print(corr.head())

plt.figure(figsize = (10,10))

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True)

In [ ]:
count_amenities = listing['amenities'].value_counts()
count_amenities